In [38]:
import tkinter as tk
import pandas as pd
import pyrsm as rsm

from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn import metrics
import nltk
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    

In [39]:
    
interaction = pd.read_csv('data/RAW_interactions.csv')
recipe = pd.read_csv("data/RAW_recipes.csv")
 

In [40]:
# drop rows with na
interaction = interaction.dropna()
# reset_index since we dropped some rows
interaction = interaction.reset_index()
# drop unneeded col
interaction = interaction.drop(columns=["index"])
# take first 100 rows for example
interaction_tmp = interaction.loc[0:99, :]
# groupby recipe_id
interaction_groupby = interaction.groupby(by=["recipe_id"])["review"].count().sort_values(ascending=False)
# transform groupbyed result to dataframe
tmp = interaction_groupby.to_frame()
tmp = tmp.rename(columns={"review": "review_count"})
# see how many recipes are there
len(tmp.query("review_count > 80")) # 1067
# if > 80, assgin 1 in the selected col
tmp["selected"] = rsm.ifelse(tmp.review_count > 80, 1, 0)
# reset_index
tmp = tmp.reset_index()
# merge
interaction_merge = interaction.merge(tmp, on="recipe_id", how="left") # left_join
# get selected data only
interaction_selected = interaction_merge.query("selected == 1")
# reset_index
interaction_selected = interaction_selected.reset_index()
# drop unneeded col
interaction_selected = interaction_selected.drop(columns=["index"])
# model to preprocessing reviews

nltk.download('punkt') # downloads you a model                                                                                                         
nltk.download('stopwords') 
ps = PorterStemmer() 
# return a list of tokens
def pre_processing_by_nltk(doc, stemming = True, need_sent = False):
    # step 1: get sentences
    sentences = sent_tokenize(doc)
    # step 2: get tokens
    tokens = []
    for sent in sentences:
        words = word_tokenize(sent)
        # step 3 (optional): stemming
        if stemming:
            words = [ps.stem(word) for word in words]
        if need_sent:
            tokens.append(words)
        else:
            tokens += words
    return [w.lower() for w in tokens if w.isalpha()]
review_split_100 = interaction_selected.iloc[:100].review.apply(pre_processing_by_nltk)
#read two txt to recognize pos or neg words
pos_words = open("positive-words.txt").read().split('\n')
neg_words = open("negative-words.txt").read().split('\n')
#function to calculate percentages
def percentage_PN(re): 
    #calculate positive word
    numPosWords = 0
    for word in re:
        if word in pos_words:
            numPosWords += 1
    
    # calculate negative words    
    numNegWords = 0   
    for word in re:
        if word in neg_words:
            numNegWords += 1
    
    #decide if positiv eor negative
    if numPosWords > numNegWords:
        return("Positive")
    elif numNegWords > numPosWords:
        return("Negative")
    elif numNegWords == numPosWords:
        return("Neutral") 
# apply on whole data
per_results = review_split_100.apply(percentage_PN)
#combine percentage results with recipe id
d = {'recipe_id':interaction.recipe_id ,'per_results':per_results}
df_per = pd.DataFrame(data = d)
df_per = df_per.dropna()
# combine num_df_per with recipe to get final dataset
df_per = df_per.rename(columns={"recipe_id": "id"})
# final dataset
DF = df_per.merge(recipe, on="id", how="left") # left_join
# specify X and y
X, y = DF.loc[:, ["minutes", "n_steps", "n_ingredients"]], DF["per_results"]
# splitting the train-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, shuffle=False)
# final model
clf_cv = MLPClassifier(
    activation="tanh",
    solver="lbfgs",
    alpha=0.01,
    hidden_layer_sizes=(4,4),
    random_state=1234,
    max_iter=10000,
).fit(X_train, y_train)

[nltk_data] Error loading punkt: <urlopen error [WinError 10061]
[nltk_data]     由于目标计算机积极拒绝，无法连接。>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10061]
[nltk_data]     由于目标计算机积极拒绝，无法连接。>


In [58]:
import tkinter as tk  
import tkinter.messagebox
import pickle
 
window = tk.Tk()

window.title('Wellcome to LogIn Website')

window.geometry('500x800') 

canvas = tk.Canvas(window, width=400, height=370, bg='grey')
image_file = tk.PhotoImage(file='picture.png')
image = canvas.create_image(200, 0, anchor='n', image=image_file)
canvas.pack(side='top')
tk.Label(window, text='Recipe Prediction Log-in',font=('Arial', 25)).pack()
tk.Label(window, text='only for UCSD usage',font=('Arial', 10)).pack()

################################################################     
######################### User log-in interface################
################################################################ 

tk.Label(window, text='User name:', font=('Arial', 14)).place(x=60, y=500)
tk.Label(window, text='Password:', font=('Arial', 14)).place(x=60, y=540)
 
var_usr_name = tk.StringVar()
var_usr_name.set('shang@ucsd.edu')
entry_usr_name = tk.Entry(window, textvariable=var_usr_name, font=('Arial', 14))
entry_usr_name.place(x=180, y=500)

var_usr_pwd = tk.StringVar()
entry_usr_pwd = tk.Entry(window, textvariable=var_usr_pwd, font=('Arial', 14), show='*')
entry_usr_pwd.place(x=180, y=540)
 
def usr_login():
    usr_name = var_usr_name.get()
    usr_pwd = var_usr_pwd.get()
    try:
        with open('usrs_info.pickle', 'rb') as usr_file:
            usrs_info = pickle.load(usr_file)
    except FileNotFoundError:
        with open('usrs_info.pickle', 'wb') as usr_file:
            usrs_info = {'admin': 'admin'}
            pickle.dump(usrs_info, usr_file)
            usr_file.close()  
    if usr_name in usrs_info:
        if usr_pwd == usrs_info[usr_name]:
            usr_predict() 
        else:
            tkinter.messagebox.showerror(message='Error, your password is wrong, try again.')
    else:  
        is_sign_up = tkinter.messagebox.askyesno('Welcome！ ', 'You have not sign up yet. Sign up now?')
        if is_sign_up:
            usr_sign_up()

################################################################     
######################### User sign up interface################
################################################################
def usr_sign_up():
    def sign_to_recipe_Website():
        
        np = new_pwd.get()
        npf = new_pwd_confirm.get()
        nn = new_name.get()
 
        with open('usrs_info.pickle', 'rb') as usr_file:
            exist_usr_info = pickle.load(usr_file)

        if np != npf:
            tkinter.messagebox.showerror('Error', 'Password and confirm password must be the same!')
        elif nn in exist_usr_info:
            tkinter.messagebox.showerror('Error', 'The user has already signed up!')
        else:
            exist_usr_info[nn] = np
            with open('usrs_info.pickle', 'wb') as usr_file:
                pickle.dump(exist_usr_info, usr_file)
            tkinter.messagebox.showinfo('Welcome', 'You have successfully signed up!')

            window_sign_up.destroy()

    window_sign_up = tk.Toplevel(window)
    window_sign_up.geometry('300x200')
    window_sign_up.title('Sign up window')
 
    new_name = tk.StringVar()
    new_name.set('shang@ucsd.edu')
    tk.Label(window_sign_up, text='User name: ').place(x=10, y=10)
    entry_new_name = tk.Entry(window_sign_up, textvariable=new_name)
    entry_new_name.place(x=130, y=10) 
 
    new_pwd = tk.StringVar()
    tk.Label(window_sign_up, text='Password: ').place(x=10, y=50)
    entry_usr_pwd = tk.Entry(window_sign_up, textvariable=new_pwd, show='*')
    entry_usr_pwd.place(x=130, y=50)
 
    new_pwd_confirm = tk.StringVar()
    tk.Label(window_sign_up, text='Confirm password: ').place(x=10, y=90)
    entry_usr_pwd_confirm = tk.Entry(window_sign_up, textvariable=new_pwd_confirm, show='*')
    entry_usr_pwd_confirm.place(x=130, y=90)
 
    btn_comfirm_sign_up = tk.Button(window_sign_up, text='Sign up', command=sign_to_recipe_Website)
    btn_comfirm_sign_up.place(x=180, y=120)
    
################################################################     
######################### User predict interface################
################################################################
def usr_predict():
    window_pred = tk.Toplevel(window)
    window_pred.geometry('500x800')

    canvas = tk.Canvas(window_pred, width=400, height=370, bg='grey')
    image_file = tk.PhotoImage(file='picture.png')
    image = canvas.create_image(200, 0, anchor='n', image=image_file)
    canvas.pack(side='top')
    tk.Label(window_pred, text='Predict now!',font=('Arial', 20)).pack()
    tk.Label(window_pred, text='only for UCSD usage',font=('Arial', 10)).pack()
    # def prediction function
    
    e1 = tk.Entry(window_pred,  font=('Arial', 15)).place(x=130, y=460) 
    e2 = tk.Entry(window_pred,  font=('Arial', 15)).place(x=130, y=490)  
    e3 = tk.Entry(window_pred, font=('Arial', 15)).place(x=130, y=520)
    
    def to_predict():   
        a = e1.get()
        a = int(a)
        b = e2.get()
        b = int(b)
        c = e3.get()
        c = int(c) 
    #input variables
        tmp.loc[0, "minutes"] = a
        tmp.loc[0, "n_steps"] = b
        tmp.loc[0, "n_ingredients"] = c
        result = clf_cv.predict(tmp)
        t.insert('insert', result)
        
    # labels for input values
    tk.Label(window_pred, text='minutes:', font=('Arial', 12)).place(x=10, y=460)    
    tk.Label(window_pred, text='n_steps:', font=('Arial', 12)).place(x=10, y=490)  
    tk.Label(window_pred, text='n_ingredients:', font=('Arial', 12)).place(x=10, y=520)  
      
    button = tk.Button(window_pred, command = to_predict, text = "predict").place(x=200, y=600)
    t = tk.Text(window_pred, height=3).place(x=0, y=660)
    # pack
    e1.pack()
    e2.pack()
    e3.pack()
    button.pack() 
    t.pack() 

################################################################           
btn_login = tk.Button(window, text='Login', command=usr_login) 
btn_login.place(x=180, y=580)
btn_sign_up = tk.Button(window, text='Sign up', command=usr_sign_up)
btn_sign_up.place(x=280, y=580)  


window.mainloop()  


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\Anaconda\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-58-20189ec8b25c>", line 47, in usr_login
    usr_predict()
  File "<ipython-input-58-20189ec8b25c>", line 144, in usr_predict
    e1.pack()
AttributeError: 'NoneType' object has no attribute 'pack'
